# 第二章 语言模型，提问范式与 Token

 - [一、环境配置](#一、环境配置)
     - [1.1 加载 API key 和一些 Python 的库。](#1.1-加载-API-key-和一些-Python-的库。)
     - [1.2 Helper function 辅助函数](#1.2-Helper-function-辅助函数)
 - [二、尝试向模型提问并得到结果](#二、尝试向模型提问并得到结果)
 - [三、Tokens](#三、Tokens)
 - [四、Helper function 辅助函数 (提问范式)](#四、Helper-function-辅助函数-(提问范式))


在本章中，我们将和您分享大型语言模型（LLM）的工作原理、训练方式以及分词器（tokenizer）等细节对 LLM 输出的影响。我们还将介绍 LLM 的提问范式（chat format），这是一种指定系统消息（system message）和用户消息（user message）的方式，让您了解如何利用这种能力。

## 一、环境配置

### 1.1 加载 API key 和一些 Python 的库。
在本课程中，为您提供了一些加载 OpenAI API key 的代码。

In [31]:
!pip install openai
!pip install langchain
!pip install --upgrade tiktoken

In [32]:
import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY']
)
deployment = "gpt-3.5-turbo"  # Typically, you would use this if specifying a particular model or deployment.
print(client.api_key[:2]) #  保证key安全，不要泄露

sk


### 1.2 Helper function 辅助函数
如果之前曾参加过《ChatGPT Prompt Engineering for Developers》课程，那么对此就相对较为熟悉。
调用该函数输入 Prompt 其将会给出对应的 Completion 。

In [33]:
def get_completion(prompt):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=deployment,                                        
        messages=messages,
        temperature=0,  # this controls the randomness of the model's output
        max_tokens=1024
    )
    return response.choices[0].message.content

# function test
get_completion("hello")

'Hello! How can I assist you today?'

## 二、尝试向模型提问并得到结果

LLM 可以通过使用监督学习来构建，通过不断预测下一个词来学习。
并且，给定一个大的训练集，有数百亿甚至更多的词，你可以创建一个大规模的训练集，你可以从一
句话或一段文本的一部分开始，反复要求语言模型学习预测下一个词是什么

LLM 主要分为两种类型：基础语言模型（Base LLM）和越来越受欢迎的指令微调语言模型（Instruction Tuned LLM）。基础语言模型通过反复预测下一个词来训练，因此如果我们给它一个 Prompt，比如“从前有一只独角兽”，它可能通过逐词预测来完成一个关于独角兽在魔法森林中与其他独角兽朋友们生活的故事。

然而，这种方法的缺点是，如果您给它一个 Prompt，比如“中国的首都是哪里？”，很可能它数据中有一段互联网上关于中国的测验问题列表。这时，它可能会用“中国最大的城市是什么？中国的人口是多少？”等等来回答这个问题。但实际上，您只是想知道中国的首都是什么，而不是列举所有这些问题。然而，指令微调语言模型会尝试遵循 Prompt，并给出“中国的首都是北京”的回答。

那么，如何将基础语言模型转变为指令微调语言模型呢？这就是训练一个指令微调语言模型（例如ChatGPT）的过程。首先，您需要在大量数据上训练基础语言模型，因此需要数千亿个单词，甚至更多。这个过程在大型超级计算系统上可能需要数月时间。训练完基础语言模型后，您会通过在一小部分示例上进行进一步的训练，使模型的输出符合输入的指令。例如，您可以请承包商帮助您编写许多指令示例，并对这些指令的正确回答进行训练。这样就创建了一个用于微调的训练集，让模型学会在遵循指令的情况下预测下一个词是什么。

之后，为了提高语言模型输出的质量，常见的方法是让人类对许多不同输出进行评级，例如是否有用、是否真实、是否无害等。然后，您可以进一步调整语言模型，增加生成高评级输出的概率。这通常使用强化学习中的人类反馈（RLHF）技术来实现。相较于训练基础语言模型可能需要数月的时间，从基础语言模型到指令微调语言模型的转变过程可能只需要数天时间，使用较小规模的数据集和计算资源。

In [34]:
response = get_completion("What is the capital of China?")
print(response)

The capital of China is Beijing.


In [35]:
response = get_completion("中国的首都是哪里？")
print(response)

中国的首都是北京。


## 三、Tokens

到目前为止对 LLM 的描述中，我们将其描述为一次预测一个单词，但实际上还有一个更重要的技术细节。即 **`LLM 实际上并不是重复预测下一个单词，而是重复预测下一个 token`** 。当 LLM 接收到输入时，它将将其转换为一系列 token，其中每个 token 都代表常见的字符序列。例如，对于 "Learning new things is fun!" 这句话，每个单词都被转换为一个 token ，而对于较少使用的单词，如 "Prompting as powerful developer tool"，单词 "prompting" 会被拆分为三个 token，即"prom"、"pt"和"ing"。

当您要求 ChatGPT 颠倒 "lollipop" 的字母时，由于分词器（tokenizer） 将 "lollipop" 分解为三个 token，即 "l"、"oll"、"ipop"，因此 ChatGPT 难以正确输出字母的顺序。您可以通过在字母之间添加连字符或空格的方式，使分词器将每个字母分解为单独的 token，从而帮助 ChatGPT 更好地认识单词中的每个字母并正确输出它们。

In [36]:
# 为了更好展示效果，这里就没有翻译成中文的 Prompt
# 注意这里的字母翻转出现了错误，吴恩达老师正是通过这个例子来解释 token 的计算方式
response = get_completion("Take the letters in lollipop \
and reverse them")
print(response)

pilpolol


"lollipop" in reverse should be "popillol"

In [37]:
response = get_completion("""Take the letters in \
l-o-l-l-i-p-o-p and reverse them""")

print(response)

p-o-p-i-l-l-o-l


![Tokens.png](/docs/figures/C2/tokens.png)

对于英文输入，一个 token 一般对应 4 个字符或者四分之三个单词；对于中文输入，一个 token 一般对应一个或半个词。

不同模型有不同的 token 限制，需要注意的是，这里的 token 限制是输入的 Prompt 和输出的 completion 的 token 数之和，因此输入的 Prompt 越长，能输出的 completion 的上限就越低。

ChatGPT3.5-turbo 的 token 上限是 4096。

## 四、Helper function 辅助函数 (提问范式)
下面是课程中用到的辅助函数。
下图是 OpenAI 提供的一种提问范式，接下来吴恩达老师就是在演示如何利用这种范式进行更好的提问
![Chat-format.png](../../figures/Chat-format.png)

System 信息用于指定模型的规则，例如设定、回答准则等，而 assistant 信息就是让模型完成的具体指令

In [38]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=500):
    '''
    封装一个支持更多参数的自定义访问 OpenAI GPT3.5 的函数

    参数: 
    messages: 这是一个消息列表，每个消息都是一个字典，包含 role(角色）和 content(内容)。角色可以是'system'、'user' 或 'assistant’，内容是角色的消息。
    model: 调用的模型，默认为 gpt-3.5-turbo(ChatGPT)，也可以选择 gpt-4
    temperature: 这决定模型输出的随机程度，默认为0，表示输出将比较确定。增加温度会使输出更随机。
    max_tokens: 这决定模型输出的最大的 token 数。
    '''
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # 这决定模型输出的随机程度
        max_tokens=max_tokens, # 这决定模型输出的最大的 token 数
    )
    content = response.choices[0].message.content


    return content

In [39]:
messages =  [  
{'role':'system', 
 'content':"""You are an assistant who\
 responds in the style of Dr Seuss."""},    
{'role':'user', 
 'content':"""write me a very short poem\
 about a happy carrot"""},  
] 
response = get_completion_from_messages(messages, temperature=1)
print(response)

Oh, the happy carrot so bright and so sweet,
In the garden it grows, where the sun and rain meet.
With a cheerful orange hue and a leafy green top,
This jolly little veggie will never stop!
It jumps and it bounces with joy each day,
The happiest carrot in every way!
So let's all smile and give a cheer,
For the joyous carrot, so full of good cheer!


In [40]:
messages =  [  
{'role':'system', 
 'content':'你是一个助理， 并以 Seuss 苏斯博士的风格作出回答。'},    
{'role':'user', 
 'content':'就快乐的小鲸鱼为主题给我写一首短诗'},  
] 
response = get_completion_from_messages(messages, temperature=1)
print(response)

在大海深处有只小鲸鱼，
快乐自在像风儿舒舒。
它跃出水面欢歌笑，
尾巴翻飞有模有样。

蓝蓝的海水是它的乐园，
游来游去从不懒惰。
和海豚一起追逐泡沫，
跳跃欢快展现活力无穷。

快乐的小鲸鱼是海洋的宝，
带来笑声让人陶醉。
在涌动的浪花中自由自在，
快乐如歌是它的标签。


In [41]:
# length
messages =  [  
{'role':'system',
 'content':'All your responses must be \
one sentence long.'},    
{'role':'user',
 'content':'write me a story about a happy carrot'},  
] 
response = get_completion_from_messages(messages, temperature =1)
print(response)

Once there was a cheerful carrot named Carl who loved making friends with all the veggies in the garden.


In [42]:
# 长度控制
messages =  [  
{'role':'system',
 'content':'你的所有答复只能是一句话'},    
{'role':'user',
 'content':'写一个关于快乐的小鲸鱼的故事'},  
] 
response = get_completion_from_messages(messages, temperature =1)
print(response)

小鲸鱼欢快地在海洋中游泳，与海洋的朋友们相处融洽，每天都过得充满快乐和幸福。


In [43]:
# combined
messages =  [  
{'role':'system',
 'content':"""You are an assistant who \
responds in the style of Dr Seuss. \
All your responses must be one sentence long."""},    
{'role':'user',
 'content':"""write me a story about a happy carrot"""},
] 
response = get_completion_from_messages(messages, 
                                        temperature =1)
print(response)

In a garden so bright, there lived a carrot named Sprite.


In [44]:
# 以上结合
messages =  [  
{'role':'system',
 'content':'你是一个助理， 并以 Seuss 苏斯博士的风格作出回答，只回答一句话'},    
{'role':'user',
 'content':'写一个关于快乐的小鲸鱼的故事'},
] 
response = get_completion_from_messages(messages, temperature =1)
print(response)

在大海深处有只小鲸鱼，欢快地跃出水面，快乐地歌唱，世界因它的存在而充满了欢乐与光芒。


In [45]:
def get_completion_and_token_count(messages, 
                                   model="gpt-3.5-turbo", 
                                   temperature=0, 
                                   max_tokens=500):
    """
    使用 OpenAI 的 GPT-3 模型生成聊天回复，并返回生成的回复内容以及使用的 token 数量。

    参数:
    messages: 聊天消息列表。
    model: 使用的模型名称。默认为"gpt-3.5-turbo"。
    temperature: 控制生成回复的随机性。值越大，生成的回复越随机。默认为 0。
    max_tokens: 生成回复的最大 token 数量。默认为 500。

    返回:
    content: 生成的回复内容。
    token_dict: 包含'prompt_tokens'、'completion_tokens'和'total_tokens'的字典，分别表示提示的 token 数量、生成的回复的 token 数量和总的 token 数量。
    """
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # 这决定模型输出的随机程度
        max_tokens=max_tokens, # 这决定模型输出的最大的 token 数
    )
    content = response.choices[0].message.content
    #print(response)
    token_dict = {
        'prompt_tokens': response.usage.prompt_tokens,
        'completion_tokens': response.usage.completion_tokens,
        'total_tokens': response.usage.total_tokens,
    }

    return content, token_dict

In [46]:
messages = [
{'role':'system', 
 'content':"""You are an assistant who responds\
 in the style of Dr Seuss."""},    
{'role':'user',
 'content':"""write me a very short poem \ 
 about a happy carrot"""},  
] 
response, token_dict = get_completion_and_token_count(messages)
print(response)

Oh, the happy carrot, so bright and so bold,
In the garden, its story is joyfully told.
With a leafy green top and a vibrant orange hue,
It dances and sings, bringing smiles to you.


In [47]:
messages =  [  
{'role':'system', 
 'content':'你是一个助理， 并以 Seuss 苏斯博士的风格作出回答。'},    
{'role':'user', 
 'content':'就快乐的小鲸鱼为主题给我写一首短诗'},  
] 
response, token_dict = get_completion_and_token_count(messages)
print(response)

在大海深处有一只小鲸鱼，
它快乐地游来游去，不停歇。
尾巴摆动，水花飞溅，
欢乐的歌声响彻海洋。

它和海豚一起跳舞，
在阳光下闪闪发亮。
小鱼儿们围着它转圈，
快乐的时光总是那么短暂。

小鲸鱼笑得灿烂如阳光，
它是大海中的快乐使者。
让我们跟随它的脚步，
一起享受生活的美好时刻。


In [48]:
print(token_dict)

{'prompt_tokens': 67, 'completion_tokens': 191, 'total_tokens': 258}


最后，我们认为 Prompt 对 AI 应用开发的革命性影响仍未得到充分重视低。在传统的监督机器学习工作流中，如果想要构建一个可以将餐厅评论分类为正面或负面的分类器，首先需要获取一大批带有标签的数据，可能需要几百个，这个过程可能需要几周，甚至一个月的时间。接着，您需要在这些数据上训练一个模型，找到一个合适的开源模型，并进行模型的调整和评估，这个阶段可能需要几天、几周，甚至几个月的时间。最后，您可能需要使用云服务来部署模型，将模型上传到云端，并让它运行起来，才能最终调用您的模型。整个过程通常需要一个团队数月时间才能完成。

相比之下，使用基于 Prompt 的机器学习方法，当您有一个文本应用时，只需提供一个简单的 Prompt 就可以了。这个过程可能只需要几分钟，如果需要多次迭代来得到有效的 Prompt 的话，最多几个小时即可完成。在几天内（尽管实际情况通常是几个小时），您就可以通过 API 调用来运行模型，并开始使用。一旦您达到了这个步骤，只需几分钟或几个小时，就可以开始调用模型进行推理。因此，以前可能需要花费六个月甚至一年时间才能构建的应用，现在只需要几分钟或几个小时，最多是几天的时间，就可以使用 Prompt 构建起来。这种方法正在极大地改变 AI 应用的快速构建方式。

需要注意的是，这种方法适用于许多非结构化数据应用，特别是文本应用，以及越来越多的视觉应用，尽管目前的视觉技术仍在发展中。但它并不适用于结构化数据应用，也就是那些处理 Excel 电子表格中大量数值的机器学习应用。然而，对于适用于这种方法的应用，AI 组件可以被快速构建，并且正在改变整个系统的构建工作流。构建整个系统可能仍然需要几天、几周或更长时间，但至少这部分可以更快地完成。

下一个章中，我们将展示如何利用这些组件来评估客户服务助手的输入。
这将是本课程中构建在线零售商客户服务助手的更完整示例的一部分。